In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import os

In [9]:
# Load in your data here
data = pd.read_csv("AEItraumahc_allPSD.csv") #file name here
print(data.shape[1])
print(data.shape[1])
print(data.iloc[:,117])

118
118
0      1
1      1
2      1
3      1
4      1
      ..
211    0
212    0
213    0
214    0
215    0
Name: Class, Length: 216, dtype: int64


In [10]:
bands = [
    ('Delta', data.iloc[:, list(range(0,22))+[-1]]),
    ('Theta', data.iloc[:, list(range(0,3)) + list(range(22,41))+[-1]]),
    ('Alpha', data.iloc[:, list(range(0,3)) + list(range(41,60))+[-1]]),
    ('Beta', data.iloc[:, list(range(0,3)) + list(range(60,79))+[-1]]),
    ('HighBeta', data.iloc[:, list(range(0,3)) + list(range(79,98))+[-1]]),
    ('Gamma', data.iloc[:, list(range(0,3)) + list(range(98,117))+[-1]]),
    ('All', data.iloc[:, 0:118])
]
all_model_data = []

#bands = [delta, theta, alpha, beta, highbeta, gamma, allb]

for label, band in bands:
    # Data preprocessing
    print(f"\n{label} columns:\n", band.columns.tolist())
    X = band.drop('Class', axis=1)
    y = band['Class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, stratify=None)
    pca = PCA(0.95)
    X_pca = pca.fit_transform(X)
    X_pca.shape
    X = X.dropna()
    X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.1, random_state=30)
    svclassifier = SVC(kernel='linear',probability=True)
    svclassifier.fit(X_train, y_train)
    y_pred = svclassifier.predict(X_test)
    print('SVM accuracy:', svclassifier.score(X_test, y_test))
    print('SVM classification report:\n', classification_report(y_test, y_pred))
    y_scores = svclassifier.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    # Calculate AUC
    svm_auc = auc(fpr, tpr)
    print('SVM AUC value:', svm_auc)
    newsvm=SVC(kernel='linear', probability=True)
    newsvm.fit(X_train_pca, y_train)
    y_pred = newsvm.predict(X_test_pca)
    print('\nSVM accuracy for PCA:', newsvm.score(X_test_pca, y_test))
    print('SVM classification report for PCA:\n', classification_report(y_test, y_pred))
    y_scores = newsvm.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcasvm_auc = auc(fpr, tpr)
    print('SVM AUC value for PCA:', pcasvm_auc)
    
    knn = KNeighborsClassifier(n_neighbors=10)
    knn.fit(X_train, y_train)
    print('KNN accuracy:', knn.score(X_test, y_test))
    y_pred = knn.predict(X_test)
    print('KNN classification report:\n',classification_report(y_test, y_pred))
    y_scores = knn.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    knn_auc = auc(fpr, tpr)
    print('KNN AUC value:', knn_auc)
    pknn = KNeighborsClassifier(n_neighbors=10)
    pknn.fit(X_train_pca, y_train)
    print('\nKNN accuracy for PCA:', pknn.score(X_test_pca, y_test))
    y_pred = pknn.predict(X_test_pca)
    print('KNN classification report for PCA\n', classification_report(y_test, y_pred))
    y_scores = pknn.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcaknn_auc = auc(fpr, tpr)
    print('KNN AUC value for PCA',pcaknn_auc)
    
    model = RandomForestClassifier(n_estimators=40)
    model.fit(X_train, y_train)
    print('Random Forest accuracy:', model.score(X_test, y_test))
    y_pred = model.predict(X_test)
    print('Random Forest classification report\n',classification_report(y_test, y_pred))
    y_scores = model.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    rf_auc = auc(fpr, tpr)
    print('Random Forest AUC value:', rf_auc)
    pmodel = RandomForestClassifier(n_estimators=40)
    pmodel.fit(X_train_pca, y_train)
    print('\nRandom Forest accuracy for PCA:', pmodel.score(X_test_pca, y_test))
    y_pred = pmodel.predict(X_test_pca)
    print('Random Forest classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = pmodel.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcarf_auc = auc(fpr, tpr)
    print('Random Forest AUC value for PCA:', pcarf_auc)
    
    lg = LogisticRegression()
    lg.fit(X_train, y_train)
    print('Logistic Regression accuracy:', lg.score(X_test, y_test))
    y_pred = lg.predict(X_test)
    print('Logistic Regression classification report:\n',classification_report(y_test, y_pred))
    y_scores = lg.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    lg_auc = auc(fpr, tpr)
    print('Logistic Regression AUC value:',lg_auc)
    plg = LogisticRegression()
    plg.fit(X_train_pca, y_train)
    print('\nLogistic Regression accuracy for PCA:',plg.score(X_test_pca, y_test))
    y_pred = plg.predict(X_test_pca)
    print('Logistic Regression classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = plg.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pcalg_auc = auc(fpr, tpr)
    print('Logistic Regression AUC value for PCA',pcalg_auc)
    
    clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
    clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    y_pred = clf.predict(X_test)
    print('XGB accuracy:',clf.score(X_test, y_test))
    print('XGB Classification report:\n',classification_report(y_test, y_pred))
    y_scores = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    xgb_auc = auc(fpr, tpr)
    print('XGB AUC value:',xgb_auc)
    pxgb = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
    pxgb.fit(X_train_pca, y_train, eval_set=[(X_test_pca, y_test)])
    print('\nXGB accuracy for PCA:', pxgb.score(X_test_pca, y_test))
    y_pred = pxgb.predict(X_test_pca)
    print('XGB classification report for PCA:\n',classification_report(y_test, y_pred))
    y_scores = pxgb.predict_proba(X_test_pca)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, y_scores)
    pxgb_auc = auc(fpr, tpr)
    print('XBG AUC for PCA:', pxgb_auc)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    model_row = [
        f"Trauma+AEI_{label}_PSD",
        svclassifier.score(X_test, y_test), svm_auc,
        knn.score(X_test, y_test), knn_auc,
        model.score(X_test, y_test), rf_auc,
        lg.score(X_test, y_test), lg_auc,
        clf.score(X_test, y_test), xgb_auc
    ]
    pca_row = [
        f"Trauma+AEI_{label}_PSDPCA", 
        newsvm.score(X_test_pca, y_test), pcasvm_auc,
        pknn.score(X_test_pca, y_test),pcaknn_auc,
        pmodel.score(X_test_pca, y_test),pcarf_auc,
        plg.score(X_test_pca, y_test),pcalg_auc,
        pxgb.score(X_test_pca, y_test), pxgb_auc
    ]
    all_model_data.append(model_row)
    all_model_data.append(pca_row)
    for row in all_model_data:
        print(row, type(row), len(row) if hasattr(row, '__len__') else 'Not iterable')
columns = ['Combination', 'SVM Accuracy', 'SVM AUC', 
           'KNN Accuracy', 'KNN AUC', 
           'Random Forest Accuracy','Random Forest AUC', 
           'Logistic Regression Accuracy', 'Logistic Regression AUC', 
           'XGB Accuracy', 'XGB AUC']
    
df = pd.DataFrame(all_model_data, columns=columns)
df.to_csv('masterdata.csv', mode='a',header=not os.path.exists('masterdata.csv'),index=False)


Delta columns:
 ['age', 'education', 'IQ', 'AB.A.delta.a.FP1', 'AB.A.delta.b.FP2', 'AB.A.delta.c.F7', 'AB.A.delta.d.F3', 'AB.A.delta.e.Fz', 'AB.A.delta.f.F4', 'AB.A.delta.g.F8', 'AB.A.delta.h.T3', 'AB.A.delta.i.C3', 'AB.A.delta.j.Cz', 'AB.A.delta.k.C4', 'AB.A.delta.l.T4', 'AB.A.delta.m.T5', 'AB.A.delta.n.P3', 'AB.A.delta.o.Pz', 'AB.A.delta.p.P4', 'AB.A.delta.q.T6', 'AB.A.delta.r.O1', 'AB.A.delta.s.O2', 'Class']
SVM accuracy: 0.5909090909090909
SVM classification report:
               precision    recall  f1-score   support

           0       0.60      0.30      0.40        10
           1       0.59      0.83      0.69        12

    accuracy                           0.59        22
   macro avg       0.59      0.57      0.54        22
weighted avg       0.59      0.59      0.56        22

SVM AUC value: 0.4833333333333334

SVM accuracy for PCA: 0.7727272727272727
SVM classification report for PCA:
               precision    recall  f1-score   support

           0       0.67      

/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


XGB accuracy: 0.5
XGB Classification report:
               precision    recall  f1-score   support

           0       0.40      0.20      0.27        10
           1       0.53      0.75      0.62        12

    accuracy                           0.50        22
   macro avg       0.46      0.47      0.44        22
weighted avg       0.47      0.50      0.46        22

XGB AUC value: 0.4708333333333333
[0]	validation_0-logloss:0.54950
[1]	validation_0-logloss:0.46991
[2]	validation_0-logloss:0.43335
[3]	validation_0-logloss:0.41151
[4]	validation_0-logloss:0.37513
[5]	validation_0-logloss:0.37440
[6]	validation_0-logloss:0.38144
[7]	validation_0-logloss:0.36505
[8]	validation_0-logloss:0.37856
[9]	validation_0-logloss:0.38691

XGB accuracy for PCA: 0.9090909090909091
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91        10
           1       1.00      0.83      0.91        12

    accuracy         

/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression classification report:
               precision    recall  f1-score   support

           0       0.17      0.10      0.12        10
           1       0.44      0.58      0.50        12

    accuracy                           0.36        22
   macro avg       0.30      0.34      0.31        22
weighted avg       0.31      0.36      0.33        22

Logistic Regression AUC value: 0.26666666666666666

Logistic Regression accuracy for PCA: 0.7727272727272727
Logistic Regression classification report for PCA:
               precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.89      0.67      0.76        12

    accuracy                           0.77        22
   macro avg       0.79      0.78      0.77        22
weighted avg       0.80      0.77      0.77        22

Logistic Regression AUC value for PCA 0.9249999999999999
[0]	validation_0-logloss:0.75315
[1]	validation_0-logloss:0.71362
[2]	validation_0-l

...

[0]	validation_0-logloss:0.74233
[1]	validation_0-logloss:0.81133
XGB accuracy: 0.45454545454545453
XGB Classification report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.50      0.83      0.62        12

    accuracy                           0.45        22
   macro avg       0.25      0.42      0.31        22
weighted avg       0.27      0.45      0.34        22

XGB AUC value: 0.325
[0]	validation_0-logloss:0.60380
[1]	validation_0-logloss:0.53916
[2]	validation_0-logloss:0.49729
[3]	validation_0-logloss:0.44825
[4]	validation_0-logloss:0.42949
[5]	validation_0-logloss:0.40685
[6]	validation_0-logloss:0.38457
[7]	validation_0-logloss:0.37831
[8]	validation_0-logloss:0.36698
[9]	validation_0-logloss:0.36266
[10]	validation_0-logloss:0.34821
[11]	validation_0-logloss:0.34702
[12]	validation_0-logloss:0.34852
[13]	validation_0-logloss:0.34481
[14]	validation_0-logloss:0.33433
[15]	validation_0-loglos

/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0]	validation_0-logloss:0.67744
[1]	validation_0-logloss:0.72926
[2]	validation_0-logloss:0.72017
XGB accuracy: 0.5454545454545454
XGB Classification report:
               precision    recall  f1-score   support

           0       0.50      0.20      0.29        10
           1       0.56      0.83      0.67        12

    accuracy                           0.55        22
   macro avg       0.53      0.52      0.48        22
weighted avg       0.53      0.55      0.49        22

XGB AUC value: 0.5958333333333333
[0]	validation_0-logloss:0.53197
[1]	validation_0-logloss:0.47013
[2]	validation_0-logloss:0.41891
[3]	validation_0-logloss:0.38986
[4]	validation_0-logloss:0.37837
[5]	validation_0-logloss:0.38034
[6]	validation_0-logloss:0.37142
[7]	validation_0-logloss:0.38163
[8]	validation_0-logloss:0.38014

XGB accuracy for PCA: 0.9090909090909091
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.83      1.00      0.91      

/home/novo/miniforge3/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[0]	validation_0-logloss:0.79907
[1]	validation_0-logloss:0.82014
XGB accuracy: 0.45454545454545453
XGB Classification report:
               precision    recall  f1-score   support

           0       0.25      0.10      0.14        10
           1       0.50      0.75      0.60        12

    accuracy                           0.45        22
   macro avg       0.38      0.42      0.37        22
weighted avg       0.39      0.45      0.39        22

XGB AUC value: 0.15416666666666665
[0]	validation_0-logloss:0.62048
[1]	validation_0-logloss:0.55607
[2]	validation_0-logloss:0.52396
[3]	validation_0-logloss:0.46693
[4]	validation_0-logloss:0.45118
[5]	validation_0-logloss:0.46048
[6]	validation_0-logloss:0.43264
[7]	validation_0-logloss:0.44641
[8]	validation_0-logloss:0.45966

XGB accuracy for PCA: 0.8181818181818182
XGB classification report for PCA:
               precision    recall  f1-score   support

           0       0.71      1.00      0.83        10
           1       1.00   